# 主要是測試各種功能用的，自行增加區塊來測試(記得加註解)

## torch測試及可用顯存測試

In [ ]:
import torch
from psutil import virtual_memory
print(torch.cuda.is_available())
GPUmem = torch.cuda.mem_get_info()
mem = GPUmem[1] /1024 / 1024 / 1024
mem += virtual_memory().total /1024 / 1024 / 1024 / 2
mem

## 功能檔案分離測試

In [ ]:
import WebDisplay
data = {
    'name': 'example',
    'value': 42,
    'status': 'active'
}
web_display = WebDisplay.WebDisplay(data)
web_display.print_data()

## 下載YouTube影片及聲音測試

In [ ]:
from bs4 import BeautifulSoup
from pytubefix import YouTube
yt = YouTube('https://music.youtube.com/watch?v=NycFr6D6DSw&si=przy_9oxIVQs5HI9')
print('download...')
yt.streams.get_audio_only().download(output_path='download/')   #下載音訊
print('ok!')

In [ ]:
acc = yt.streams.get_audio_only()
acc.download(output_path='download/')   #下載音訊

### 加入手動輸入連結

In [6]:
url = input()

In [7]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation
import os
vedio = YouTubeDownload.download_youtube_video(url)
audio = YouTubeDownload.download_youtube_audio(url)
separation_audio = MusicSeparation.run_separation(audio)
merge = YouTubeDownload.merge_video_audio(vedio, separation_audio)
print(vedio, audio , merge, separation_audio)
#os.system(merge)
separation_audio

已儲存：h:\NFU\score\score/output\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-vocals.wav
已儲存：h:\NFU\score\score/output\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-other.wav
"h:\NFU\score\download\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】.mp4" "h:\NFU\score\download\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】.m4a" "h:\NFU\score\download\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】_merged.mp4" h:\NFU\score\score/output\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-other.wav


'h:\\NFU\\score\\score/output\\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】\\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-other.wav'

In [3]:
os.system(merge)
audio

'"h:\\NFU\\score\\download\\【MUTA 嫣汐】《嫣橙色》PV【Yunomi】.m4a"'

### 整合ASR辨識

In [8]:
MusicSeparation.run_ASR(separation_audio)

選擇的檔案: h:\NFU\score\score/output\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-other.wav
偵測語言: ja
已輸出至資料夾:h:\NFU\score\score/output\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】


'h:\\NFU\\score\\score/output\\Self Control by amatsukauto ໒꒱· ﾟ 【orignal】Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-other.srt'

## NeMo測試(確認無用)

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

print(nemo.__version__)


## 字幕辨識測試

In [ ]:
import whisper
whisper.available_models()

In [3]:
import os
import torch
import tkinter as tk
from tkinter import filedialog
import whisper

#建立Tk主視窗，但隱藏
root = tk.Tk()
root.withdraw()
root.attributes('-topmost', True)  #對話框出現在最前方

#選擇音訊檔案
current_dir = os.getcwd()
file_path = filedialog.askopenfilename(
    title="請選擇音訊檔案",
    initialdir=current_dir,
    filetypes=[("音訊檔案", "*.wav *.mp3 *.m4a *.flac"), ("所有檔案", "*.*")]
)

if not file_path:
    print("未選擇檔案")
    exit()

print("選擇的檔案:", file_path)

#載入Whisper模型
if(torch.cuda.is_available()):
    model = whisper.load_model("large-v3", device="cuda")
else:
    model = whisper.load_model("large-v3", device="cpu")

#轉錄音訊，Whisper會自動偵測語言
result = model.transcribe(file_path)
print("偵測語言:", result["language"])

#輸出資料夾:與音訊檔同一層
audio_dir = os.path.dirname(file_path)

#取得音訊檔名稱(不含副檔名)
filename = os.path.splitext(os.path.basename(file_path))[0]

#---SRT---
with open(os.path.join(audio_dir, f"{filename}.srt"), "w", encoding="utf-8") as f:
    for i, seg in enumerate(result["segments"], start=1):
        text = seg['text'].strip()
        if not text:
            continue
        f.write(f"{i}\n")
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02},{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02},{int((end%1)*1000):03}"
        f.write(f"{start_time} --> {end_time}\n")
        f.write(f"{text}\n\n")

#---VTT---
with open(os.path.join(audio_dir, f"{filename}.vtt"), "w", encoding="utf-8") as f:
    f.write("WEBVTT\n\n")
    for seg in result["segments"]:
        text = seg['text'].strip()
        if not text:
            continue
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02}.{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02}.{int((end%1)*1000):03}"
        f.write(f"{start_time} --> {end_time}\n")
        f.write(f"{text}\n\n")

#---TXT---
with open(os.path.join(audio_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
    for seg in result["segments"]:
        text = seg['text'].strip()
        if not text:
            continue
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02}.{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02}.{int((end%1)*1000):03}"
        f.write(f"[{start_time} --> {end_time}] {text}\n")

print(f"已輸出至資料夾:{audio_dir}")


選擇的檔案: H:/NFU/score/score/output/TNT时代少年团 X 优酸乳《Lemon》认人歌词版 CN_PIN_ENG｜嚼柠檬主题曲中文版/TNT时代少年团 X 优酸乳《Lemon》认人歌词版 CN_PIN_ENG｜嚼柠檬主题曲中文版-vocals.wav
偵測語言: zh
已輸出至資料夾:H:/NFU/score/score/output/TNT时代少年团 X 优酸乳《Lemon》认人歌词版 CN_PIN_ENG｜嚼柠檬主题曲中文版


### 分離功能至MusicSeparation.py內測試

In [1]:
from MusicSeparation import MusicSeparation
#MusicSeparation.run_ASR(r"H:/NFU/score/score/output/Immortals/Immortals-vocals.wav")    #若無傳入值則由使用者手動選取
MusicSeparation.run_ASR()

選擇的檔案: H:/NFU/score/score/output/Self Control by amatsukauto ໒꒱· ﾟ 【orignal】/Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-vocals.wav
偵測語言: ja
已輸出至資料夾:H:/NFU/score/score/output/Self Control by amatsukauto ໒꒱· ﾟ 【orignal】


'H:/NFU/score/score/output/Self Control by amatsukauto ໒꒱· ﾟ 【orignal】Self Control by amatsukauto ໒꒱· ﾟ 【orignal】-vocals.srt'